In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

### データの用意
pandasを用いてデータを用意する
使うデータは [International Airport Passengers](https://datamarket.com/data/set/22u3/international-airline-passengers-monthly-totals-in-thousands-jan-49-dec-60#!ds=22u3&display=line) 

In [ ]:
df = pd.read_csv('AirportPassengers.csv',delimiter=";").dropna(how="any")
# df.Passengers
from datetime import datetime
def str2time(string):
    return datetime.strptime(string, '%Y-%m')
df.index = df.Month.apply(str2time)
df = df.drop(["Month"],axis=1)
df.head()

### 時系列の定常性チェック
- 平均や分散がずっと一定なら**starionary(定常)**といえる
- 多くの時系列モデルは**定常性**を仮定している

定常性というのは厳格な基準があるが、実務上では下記の要素を定常性の条件としてもよいだろう
- 一定の平均
- 一定の分散
- 時間に依存しない自己共分散


In [ ]:
plt.plot(df.index,df.Passengers)

**増加傾向**にあることがわかる。これだと平均や分散が一定とならないように見える。
ちゃんと調べるために、下記の処理を施してみる

1. moving averageやmoving distanceをとってみる(sliding window)今回は、季節変動的なのも吸収したいから12か月分
1. Dickkey Fuller Testを用いる。定常性の統計検定である。[詳細](https://ja.wikipedia.org/wiki/%E3%83%87%E3%82%A3%E3%83%83%E3%82%AD%E3%83%BC%E2%80%93%E3%83%95%E3%83%A9%E3%83%BC%E6%A4%9C%E5%AE%9A)


DF検定には、**`statsmodels`**という時系列用統計ライブラリを使う
- そのままdfを入れるとエラーになるので、1d arrayを使う [(参考)]( https://stackoverflow.com/questions/43100441/i-am-trying-to-run-dickey-fuller-test-in-statsmodels-in-python-but-getting-error
) 

- DF検定の結果の見方がいまいち不明なので、後で調べる。


In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=12).mean()
    rolstd = timeseries.rolling(window=12).std()

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
#     dftest = adfuller(timeseries, autolag='AIC')
    dftest = adfuller(timeseries)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
array = df.Passengers.values
test_stationarity(df.Passengers)


### 時系列データを定常状態にしたい
**トレンド**と**季節変動**をうまく取り除き、定常状態にしてやりたい。
#### トレンドを予測して消す
1. positiveトレンドのときは、大きい値にペナルティを与えるような変換をかける(累乗根、log)など
1. トレンド成分を取り出す
    - Aggregation : 週/月ごとにまとめた平均を取る
    - Rolling : windowをスライドさせながら平均を取る(移動平均)
    - Polynomial Fitting : 回帰でフィッティングさせる
1. Originalからトレンド成分を引く
1. テストをしてみる。

> Also, the test statistic is smaller than the 5% critical values so we can say with 95% confidence that this is a stationary series.

らしい

In [ ]:
# 1. log
df["passengers_log"] = np.log(df.Passengers)
plt.plot(df.index, df.passengers_log)
# df["passengers_sqrt"] = np.sqrt(df.Passengers)
# plt.plot(df.index, df.passengers_sqrt)
# 2. eliminate noise
df["moving_ave"] = df.passengers_log.rolling(window=12).mean()
# df["moving_medi"] = df.passengers_log.rolling(window=12).median()
df["log_ave_diff"] = df.passengers_log - df.moving_ave
plt.plot(df.index, df.moving_ave)
# plt.plot(df.index, df.moving_medi)
plt.figure()
plt.plot(df.index, df.log_ave_diff)
timeseries = df.log_ave_diff
test_stationarity(timeseries.dropna())

※ このトレンド除去の問題点は、**window幅に依存する**ところ。株価とかだったらどういう窓のサイズを使えばいいか微妙なところ。そこで、

>
-  weighted moving averageというのを使う。
- 最近の値ほど大きな重み付けがされる
- ポピュラーなものに**exponentially weighted moving average**がある

> This TS has even lesser variations in mean and standard deviation in magnitude. Also, the test statistic is smaller than the 1% critical value, which is better than the previous case. Note that in this case there will be no missing values as all values from starting are given weights. So it’ll work even with no previous values.

 

In [ ]:
df["expweighted_ave"] = df.passengers_log.ewm(halflife=12).mean() #半減の時間
plt.plot(df.index, df.passengers_log,label="original")
plt.plot(df.index, df.moving_ave,label="moving average")
plt.plot(df.index, df.expweighted_ave,label="exponentially weighted moving average")
plt.legend()
plt.figure()
timeseries = df.passengers_log - df.expweighted_ave
test_stationarity(timeseries)

### Seasonalityを消したい
- Differencing : 特定のタイムラグを持たせて引く : df.shift()
- Decomposition : trendとseasonalityをモデル化して一気に消す : statsmodels.tsa.seasonal.seasonal_deconpose

In [ ]:
df.head()
# differencing
df["log_diff"] = df.passengers_log - df.passengers_log.shift()
plt.plot(df.index, df.log_diff)
ts = df.log_diff.dropna()
test_stationarity(ts)


In [ ]:
# Decomposing
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(df.passengers_log)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid #何コレ

plt.subplot(411)
plt.plot(df.passengers_log, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()
plt.figure()


### 時系列のその後を予測する

予測によく使われるARIMA(Auto-Regressive Integrated Moving Average)の説明と、実際に使ってみる。
ARIMAは、ARモデルと、MAモデルを統合したモデルとなっている。

ARIMAには、下記のパラメータを用いる

- `p` (number of AR terms) : 自己回帰の窓幅を何個前まで取るか
- `q` (number of moving average) : 移動平均の窓幅を何個前まで取るか
- `d` (number of differences) : 一次の差分か二次の差分か
p=0だとMA、q=0だとARモデルとできる。

pとqを決めるための大事な量が**ACF(Autoorrelation Function**と**PACF(Partial ACF)**

- p : **ACF**のupper confidential intervalがクロスするラインを選ぶ
- q : **PACF**upper confidential intervalがクロスするラインを選ぶ

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
ts = df.log_diff.dropna()
lag_acf = acf(ts, nlags=20)
lag_pacf = pacf(ts, nlags=20, method="ols")
#Plot ACF: 
plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')
#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In [ ]:
# ARIMA model
from statsmodels.tsa.arima_model import ARIMA
ts = df.log_diff.dropna()
model1 = ARIMA(ts, order=(2, 1, 0))  
results_AR = model1.fit(disp=-1)  
model2 = ARIMA(ts, order=(0, 1, 2))
results_MA = model2.fit(disp=-1)
model3 = ARIMA(ts,order=(2, 1, 2))
results_ARIMA = model3.fit()

plt.plot(ts)
plt.plot(results_AR.fittedvalues,"--", color='red',alpha=0.8,label="AR")
plt.plot(results_MA.fittedvalues,"--", color="green",alpha=0.8,label="MA")
plt.plot(results_ARIMA.fittedvalues,lw=2, color="blue",label="ARIMA")
plt.xlim("1950","1960")
plt.legend()
# plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-ts)**2))

In [ ]:
def predict(name, results):
    df[name] = pd.Series(results.fittedvalues)
    df["predict_cumsum"] = df[name].cumsum()
    # 元に戻す
    df[name] = df.predict_cumsum.fillna(0)+df.passengers_log
    df[name] = np.exp(df[name])
    df.head()
    plt.plot(df.index, df.Passengers, label="Original")
    plt.plot(df.index, df[name],color="orange",label=name)
    plt.legend()
    plt.show()
predict("ARIMA", results_ARIMA)
predict("MA", results_MA)
predict("AR",results_AR)

### 評価する

In [ ]:
df[["Passengers","ARIMA","MA","AR"]].head()

In [ ]:
#評価値
from sklearn.metrics import mean_squared_error
print("ARIMA:", np.sqrt(mean_squared_error(df.Passengers, df.ARIMA)))
print("MA", np.sqrt(mean_squared_error(df.Passengers, df.MA)))
print("AR", np.sqrt(mean_squared_error(df.Passengers, df.AR)))